In [4]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import os
import sys
import pandas as pd
import pickle
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [5]:
from batch_sv_system import get_embeds, cosine_sim, compute_plda_score

In [7]:
from utils import key2df, get_id2idx

embed_dir = "embeddings/voxc2_fbank64_voxc2untied_300f_xvector/"
si_embeds = np.load(embed_dir+"ln_lda_si_embeds.npy")
sv_embeds = np.load(embed_dir+"ln_lda_sv_embeds.npy")
si_keys = pickle.load(open(embed_dir + "/si_keys.pkl", "rb"))
sv_keys = pickle.load(open(embed_dir + "/sv_keys.pkl", "rb"))
sv_id2idx = get_id2idx(sv_keys)
si_df = key2df(si_keys)

In [ ]:
si_embed_dir = "embeddings/voxc2_fbank64_voxc2untied_300f_xvector/"
si_embeds = np.load(si_embed_dir + "/si_embeds.npy")
si_keys = pickle.load(open(si_embed_dir + "/si_keys.pkl", "rb"))
si_id2idx = get_id2idx(si_keys)
si_df = key2df(si_keys)

sv_embed_dir = "embeddings/voxc2_fbank64_voxc2untied_300f_xvector/"
sv_embeds = np.load(sv_embed_dir + "/sv_embeds.npy")
sv_keys = pickle.load(open(sv_embed_dir + "/sv_keys.pkl", "rb"))
sv_id2idx = get_id2idx(sv_keys)
sv_df = key2df(sv_keys)

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
si_embed_mean = si_embeds.mean(0)
centered_si_embeds = si_embeds - si_embed_mean.reshape(1, -1)
centered_sv_embeds = sv_embeds - si_embed_mean.reshape(1, -1)

clf = LDA(solver='svd', n_components=200)
clf.fit(centered_si_embeds, si_df.label)

lda_si_embeds = clf.transform(centered_si_embeds)
lda_sv_embeds = clf.transform(centered_sv_embeds)

# length normalization
ln_lda_si_embeds = lda_si_embeds * np.sqrt(lda_si_embeds.shape[1]) / \
                                           np.linalg.norm(lda_si_embeds, axis=1, keepdims=True)
ln_lda_sv_embeds = lda_sv_embeds * np.sqrt(lda_sv_embeds.shape[1]) / \
                                           np.linalg.norm(lda_sv_embeds, axis=1, keepdims=True)

In [ ]:
# np.save(si_embed_dir+"ln_lda_si_embeds.npy", ln_lda_si_embeds)
# np.save(sv_embed_dir+"ln_lda_sv_embeds.npy", ln_lda_sv_embeds)

In [4]:
from ioffe_plda.verifier import Verifier
py_plda_model = Verifier()
py_plda_model.fit(si_embeds, si_df.label)
# pickle.dump(py_plda_model, open("py_plda_model_ln_lda.pkl", "wb"))